# Implementing the cropping function

## The various sub-tasks involved are :

* Get shape of the big image : (p,q)
* Define the size of the tile : (a,b) = (50,50)
* Determine the number of useful tiles; ensure all tiles are of equal size (a,b) and discard the ones that arent - Might lose a few due to boundary conditions but thats okay!
* Carry out Cropping of the tiles sequentially! 
* Obtain the vertex of the each of the cropped tiles.
* Save each of these cropped tiles in respective folder.

In [12]:
#Import Stuff 
import os
import cv2
import math
import numpy as np
import pandas as pd

In [15]:
#Read the image
#source_path = '/Users/keshavaprasad/Desktop/image/MG48_3day_bs/testingcrop/S051_00.tif'
#source_path = '/Users/keshavaprasad/Desktop/image/S25/S25_00.tif'
source_path='//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/Image/pia_only/S051.tif'
image=cv2.imread(source_path)

In [16]:
# Image Basics
print(image.shape)
# Get Shape
p = image.shape[0]# p
q = image.shape[1]#q
print("Height:",p,"Width:",q)

# Defining the dimensions of a tile 
#a = b = 50
a = b = 70
#************************
cropped_tiles = [] #list for storing all cropped tile images
vertices = [] #list for storing lists of vertex values # Defining vertices of tiles for cropping
ctc = []#list for storing lists of tile numbers
#cropped_tile_list = []
background_cropped_tiles = [] #list for storing background tile images
# Cropping image Sequentially
i = j = cropped_tile_counter = correct_tile_counter = wrong_dim_tile_counter = correct_dim_tile_counter= 0


AttributeError: 'NoneType' object has no attribute 'shape'

In [4]:
# Carry oout the sequential cropping store data into respective lists
for i in range(0,math.floor(p),a):#range(start, stop, step) # to get the resolution right!
    for j in range(0,math.floor(q),b):
        #Defining the cropping box
        #tile = image[j:j+b,i:i+a]#y1: y2, x1: x2
        y1 = j
        x1 = i
        y2 = j+b
        x2 = i+a
        
        tile = image[x1:x2,y1:y2]# making crops of each subimage and then storing in tile variable and then appended to cropped_tiles list
        vertex = [x1,x2,y1,y2] # list of vertex values of each tiles
        cropped_tile_counter +=1
        
        # Calculating mean for each tile and
        #mean = np.mean(tile)
        
        # checking for tiles with wrong dimensions
        #if tile.shape[0]!=50 or tile.shape[1]!= 50:
        if tile.shape[0]!=70 or tile.shape[1]!= 70:
            wrong_dim_tile_counter += 1
        
        # Keeping only tiles with correct dimensions # Append the vertex for each of these tiles into vertices list 
        # Append the correct_tile_counter variable into ctc list
        
        #if tile.shape[0] == 50 and tile.shape[1] == 50:
        if tile.shape[0] == 70 and tile.shape[1] == 70 and not np.all((np.array(tile) == 0)):
            correct_dim_tile_counter +=1
           
            correct_tile_counter +=1
            #cropped_tiles.append(tile)
            cropped_tiles.append(cropped_tile_counter)
            ctc.append(correct_tile_counter)
            #ctc.append([cropped_tile_counter,correct_tile_counter])
            #print("correct_tile_counter:",correct_tile_counter,"with vertex:",vertex)
            vertices.append(vertex)                        
### THE TWO KEY TAKEAWAYS:number of croppedtiles and respective vertex of each cropped tile

In [5]:
print(len(cropped_tiles))

4833


In [6]:
print(len(ctc))

4833


In [7]:
# Calling DataFrame constructor after zipping # both lists, with columns specified 
#cropped_data = pd.DataFrame(list(zip(ctc, vertices)),columns =['Tile_Counter', 'Vertices']) 
cropped_data = pd.DataFrame(list(zip(cropped_tiles,ctc, vertices)),columns =['Cropped_Tile_Counter','Correct_Tile_Counter', 'Vertices']) 
#cropped_data

In [8]:
cropped_data

,Cropped_Tile_Counter,Correct_Tile_Counter,Vertices
0,9283,1,"[4340, 4410, 7420, 7490]"
1,9284,2,"[4340, 4410, 7490, 7560]"
2,9285,3,"[4340, 4410, 7560, 7630]"
3,9286,4,"[4340, 4410, 7630, 7700]"
4,9287,5,"[4340, 4410, 7700, 7770]"
...,...,...,...
4828,18578,4829,"[8750, 8820, 5390, 5460]"
4829,18579,4830,"[8750, 8820, 5460, 5530]"
4830,18580,4831,"[8750, 8820, 5530, 5600]"
4831,18581,4832,"[8750, 8820, 5600, 5670]"


In [ ]:
#cropped_data.loc[cropped_data['Tile_Counter'] == 21555] #searching rows based on specific values
#cropped_data.loc[cropped_data['mean_values'] > 15] 

In [9]:
import pickle
with open('global_xy_MG48_3day_bs_S51_bigtile_piamasked.pkl', 'wb') as g:
    pickle.dump(cropped_data, g)

In [10]:
print("Total number of tiles:",cropped_tile_counter)
print("Wasted tiles due boundary conditions:",(wrong_dim_tile_counter/cropped_tile_counter)*100,"%")
print("Useless background tiles:",(correct_dim_tile_counter - correct_tile_counter)/cropped_tile_counter*100,"%")
print("Useful tiles:",(correct_tile_counter/cropped_tile_counter)*100,"%")


Total number of tiles: 20276
Wasted tiles due boundary conditions: 1.4006707437364372 %
Useless background tiles: 0.0 %
Useful tiles: 23.836062339711976 %


## Saving useful tiles that are sequentially cropped

In [ ]:

#Dest_Path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/testingcrop/S052/'
Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images/S51/'
#Dest_Path = '/Users/keshavaprasad/Desktop/image/S25/'

# Delete all exiting images
import glob
os.chdir(Dest_Path)
files=glob.glob('*.tif')
for filename in files:
    os.unlink(filename)
# Saving the tiles in a respective folder    
for k in range(len(cropped_tiles)):
        pass
        #print("K:",k)
        #cv2.imwrite(str(k)+ ".tif", tiles[k])
        cv2.imwrite(os.path.join(Dest_Path,str(k)+".tif"), cropped_tiles[k][1])